In [ ]:
import pandas as pd
import numpy as np

| Ime spremenljivke | Pomen / opis | Tip podatka |
|---|---|---|
| customerID | Enolična identifikacijska oznaka posamezne stranke. | Kategorna (ID string) |
| gender | Spol stranke (Male ali Female). | Kategorna (nominalna) |
| SeniorCitizen | Označuje, ali je stranka starejša oseba (1 = da, 0 = ne). | Numerična (binarna, lahko tudi kategorialna) |
| Partner | Ali ima stranka partnerja (Yes / No). | Kategorna (nominalna) |
| Dependents | Ali ima stranka odvisne člane (otroke, družinske člane) (Yes / No). | Kategorna (nominalna) |
| tenure | Število mesecev, kolikor je stranka naročena pri podjetju. | Numerična (diskretna) |
| PhoneService | Ali stranka uporablja telefonske storitve (Yes / No). | Kategorna (nominalna) |
| MultipleLines | Ali ima stranka več telefonskih linij (Yes, No, No phone service). | Kategorna (nominalna) |
| InternetService | Vrsta internetne storitve (DSL, Fiber optic, No). | Kategorna (nominalna) |
| OnlineSecurity | Dodatna storitev spletne varnosti (Yes, No, No internet service). | Kategorna (nominalna) |
| OnlineBackup | Storitve spletne varnostne kopije (Yes, No, No internet service). | Kategorna (nominalna) |
| DeviceProtection | Zaščita naprav (Yes, No, No internet service). | Kategorna (nominalna) |
| TechSupport | Tehnična podpora (Yes, No, No internet service). | Kategorna (nominalna) |
| StreamingTV | Storitve spletnega televizijskega pretakanja (Yes, No, No internet service). | Kategorna (nominalna) |
| StreamingMovies | Storitve spletnega pretakanja filmov (Yes, No, No internet service). | Kategorna (nominalna) |
| Contract | Vrsta pogodbe: Month-to-month, One year, Two year. | Kategorna (ordinalna) |
| PaperlessBilling | Ali stranka uporablja brezpapirno obračunavanje (Yes / No). | Kategorna (nominalna) |
| PaymentMethod | Način plačila (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)). | Kategorna (nominalna) |
| MonthlyCharges | Mesečni strošek naročnine (v USD). | Numerična (kontinuirna) |
| TotalCharges | Skupni strošek, ki ga je plačala stranka do danes. | Numerična (kontinuirna) |
| Churn | Ali je stranka prekinila naročnino (Yes / No). | Ciljna spremenljivka, kategorialna (binarna) |

Pregled in čiščenje podatkov